In [1]:
import os
import pickle
import os.path as osp

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [26]:

methods = ['lasso', 'rs']
input_dims = [100, 1000, 10000, 100000]
threshold = [0.5, 0.9, 0.99]

zero_rate = 'zero_rate6'

In [28]:

def read_lasso_sparsity_and_time(file):
    df = pd.read_csv(file)
    return df.zero_rate6.tolist(), df.time.tolist()


In [41]:
from collections import defaultdict


def load_rs_folder(folder, target_sparsities):
    df = pd.read_csv(osp.join(folder, 'metrics.csv'))
    times = df.time.tolist()
    time_by_threshold = defaultdict(list)

    with open(osp.join(folder, 'metrics_traject.pickle'), 'rb') as f:
        alpha_dict_of_metric_list = pickle.load(f)
    for ialpha, k in enumerate(sorted(alpha_dict_of_metric_list.keys())):
        total_time = times[ialpha]
        target_zero_rate = target_sparsities[ialpha]
        metric_list = alpha_dict_of_metric_list[k]
        total_steps = len(metric_list)

        threshold_reach = [False for t in threshold]
        for i, metrics in enumerate(metric_list):
            for it, t in enumerate(threshold):
                if metrics[zero_rate] > t * target_zero_rate and (threshold_reach[it] == False):
                    t_time = (i+1) / total_steps * total_time
                    time_by_threshold[t].append(t_time)
                    threshold_reach[it] = True
        time_by_threshold['final'].append(total_time)

    return time_by_threshold


In [45]:
folder="output/HighDimLinearRegression"

data = defaultdict(list)

for idim in input_dims:
    data['x_dim'].append(idim)

    lasso_file = osp.join(folder, f"{idim}_1_lasso", "metrics.csv")
    lasso_sparsities, lasso_times = read_lasso_sparsity_and_time(lasso_file)
    print(lasso_sparsities, lasso_times)
    rs_folder = osp.join(folder, f"{idim}_1_rs")
    time_by_threshold = load_rs_folder(rs_folder, lasso_sparsities)
    print(time_by_threshold)
    for i in range(1, 5):
        data[f'Lasso_time_alpha{i}'].append(lasso_times[i])
    for t in threshold + ['final']:
        for i in range(1, 5):
            try:
                data[f'RS_time_alpha{i}_{t}lasso_zr'].append(time_by_threshold[t][i])
            except IndexError:
                data[f'RS_time_alpha{i}_{t}lasso_zr'].append(None)

[0.0, 0.44, 0.75, 0.92, 1.0] [0.0080358982086181, 0.0032978057861328, 0.0023164749145507, 0.00185227394104, 0.0016694068908691]
defaultdict(<class 'list'>, {'final': [38.97254800796509, 23.944467306137085, 24.831671476364136, 23.63450503349304, 23.910090446472168], 0.5: [4.02267050743103, 2.0610287325382233, 1.2289942617416378, 0.836853165626526], 0.9: [14.318791449069977, 5.301561860203742, 4.017865855693817, 1.9008521904945375], 0.99: [10.831270972251893]})
[0.0, 0.797, 0.946, 0.99, 0.998] [0.4090862274169922, 0.0273728370666503, 0.0251085758209228, 0.0238099098205566, 0.0232532024383544]
defaultdict(<class 'list'>, {0.5: [8.573467780351638, 2.1628575682640077, 0.907185362100601, 0.5101609654426575, 0.3647474546432495], 0.9: [8.573467780351638, 9.855895864963532, 2.100196249246597, 0.9352951033115386, 0.5534099311828612], 0.99: [8.573467780351638, 13.893605956554412, 3.048819103002548, 1.257749843597412], 'final': [34.99374604225159, 25.902485847473145, 24.85439348220825, 24.29337930

In [46]:
df = pd.DataFrame(data)

In [47]:
df

,x_dim,Lasso_time_alpha1,Lasso_time_alpha2,Lasso_time_alpha3,Lasso_time_alpha4,RS_time_alpha1_0.5lasso_zr,RS_time_alpha2_0.5lasso_zr,RS_time_alpha3_0.5lasso_zr,RS_time_alpha4_0.5lasso_zr,RS_time_alpha1_0.9lasso_zr,...,RS_time_alpha3_0.9lasso_zr,RS_time_alpha4_0.9lasso_zr,RS_time_alpha1_0.99lasso_zr,RS_time_alpha2_0.99lasso_zr,RS_time_alpha3_0.99lasso_zr,RS_time_alpha4_0.99lasso_zr,RS_time_alpha1_finallasso_zr,RS_time_alpha2_finallasso_zr,RS_time_alpha3_finallasso_zr,RS_time_alpha4_finallasso_zr
0,100,0.003298,0.002316,0.001852,0.001669,2.061029,1.228994,0.836853,NaN,5.301562,...,1.900852,NaN,NaN,NaN,NaN,None,23.944467,24.831671,23.634505,23.910090
1,1000,0.027373,0.025109,0.023810,0.023253,2.162858,0.907185,0.510161,0.364747,9.855896,...,0.935295,0.553410,13.893606,3.048819,1.25775,None,25.902486,24.854393,24.293379,25.154997
2,10000,1.379982,0.620956,0.417409,0.356211,2.982176,1.398869,0.910485,0.651693,9.212795,...,2.499372,1.701643,27.837495,17.816949,9.25042,None,35.502099,34.971727,35.705308,36.205165
3,100000,44.093867,23.535544,15.295118,11.510065,11.437851,7.760439,5.659702,NaN,23.864565,...,11.517990,NaN,43.688929,NaN,NaN,None,68.597880,65.924213,67.776763,66.195346
